<a href="https://colab.research.google.com/github/k-farruh/otus-demo/blob/main/my_first_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Этот блокнот содержит последовательность действие для обучения глубокому обучению, Apache MXNet и Gluon интерфейс. Цель - использовать сильные стороны Jupyter Notebook для представления прозы, графики, уравнений и кода в одном месте. 

In [6]:
# Устанавливаем mxnet библиотеку
!pip install mxnet

In [7]:
# Импортируем нужные библиотеки
from __future__ import print_function
import mxnet as mx
import numpy as np
from mxnet import nd, autograd, gluon
# В зависимости можете выбрать CPU или GPU
ctx = mx.cpu()
# ctx = mx.gpu()
mx.random.seed(1)

ModuleNotFoundError: ignored